In [11]:
import ee, geemap, numpy as np, pandas as pd
import geopandas as gpd
from shapely.geometry import box
import tensorflow as tf

python(34383) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


### Trying to make a Prediction Heatmap

In [3]:
ee.Initialize(project='stat-288-final-project-458620')

In [5]:
# A) Define Lagos via a simple bounding‐box (or load a GeoJSON/shapefile)
lagos_bbox = box(3.0, 6.4, 3.5, 6.8)  # [min_lon, min_lat, max_lon, max_lat]
lagos_gdf  = gpd.GeoDataFrame({'geometry':[lagos_bbox]}, crs="EPSG:4326")

# B) Make an EE FeatureCollection of patch‐centers on a 32×32 km grid
STRIDE_KM = 8
STEP_DEG  = STRIDE_KM * (1/111.0)
xmin, ymin, xmax, ymax = lagos_bbox.bounds
xs = np.arange(xmin + STEP_DEG/2, xmax, STEP_DEG)
ys = np.arange(ymin + STEP_DEG/2, ymax, STEP_DEG)

centers = []
for x in xs:
    for y in ys:
        centers.append(ee.Feature(ee.Geometry.Point([x, y])))

fc_centers = ee.FeatureCollection(centers)
print("Total patches:", fc_centers.size().getInfo())


Total patches: 42


In [12]:
# reuse your `weekly_collection` and `to_array` helpers
AOD   = ee.ImageCollection("MODIS/061/MCD19A2_GRANULES")
NDVI  = ee.ImageCollection("MODIS/MCD43A4_006_NDVI")
VNTL  = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG')
BANDS = ['Optical_Depth_047','AOD_QA','Column_WV','NDVI']

def weekly_image(start):
    start = ee.Date(start); end = start.advance(7,'day')
    aod  = AOD.filterDate(start,end).select(BANDS[:3]).median()
    nd   = NDVI.filterDate(start,end).select('NDVI').median()
    base = aod.addBands(nd)
    nm   = VNTL.filterDate(start,start.advance(1,'month')).first()
    img  = ee.Image(ee.Algorithms.If(nm, base.addBands(nm.select('avg_rad').rename('NTL')), base))
    return img.set('system:time_start', start.millis())

def to_array(img):
    return img.neighborhoodToArray(ee.Kernel.square(STRIDE_KM//2,'pixels'))

ic_arrays = ee.List.sequence(0, 1).map(  # for illustration, just 1 week; swap to your date range
    lambda i: to_array(weekly_image('2023-01-01'))
)
ic_arrays = ee.ImageCollection(ic_arrays)

In [13]:
# Sample that one-week image at every center point
samples = ic_arrays.first().sampleRegions(
    collection = fc_centers,
    scale      = 1000,
    geometries = True
)

# Pull into Python as a GeoPandas DataFrame
url        = samples.getDownloadURL(filetype='geojson')  # Pass as string parameter, not dictionary
patch_gdf  = gpd.read_file(url)  # has geometry, and array‐bands as a long flattened list

In [18]:
# A) Load your trained model
model = tf.keras.models.load_model('models/delhi_bangkok_pm25_cnn_model_v2.keras')

# B) Convert each EE‐array into a (32,32,nb) patch
def unpack_patch(row):
    vals = np.array(row['array'])      # `array` is your EE flattened band‐stack
    nb   = int(vals.size / (STRIDE_KM*STRIDE_KM))
    return vals.reshape(STRIDE_KM, STRIDE_KM, nb)

patch_gdf['patch'] = patch_gdf.apply(unpack_patch, axis=1)

# C) Stack into a single tensor
X_all = np.stack(patch_gdf['patch'].values).astype('float32') / 100.0
#    ← scale if needed

# D) Run model
preds = model.predict(X_all).ravel()
patch_gdf['pm25_pred'] = preds

KeyError: 'array'